In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings  
import time
import matplotlib.pyplot as plt
import matplotlib
import os

In [9]:
family_to_train = 'R_Addition_MultipleBond'
command = f'scp ./{family_to_train}_retraining/groups.py ./{family_to_train}_retraining/rules.py /home/khalil.nor/Code/RMG-database/input/kinetics/families/{family_to_train}'
os.system(command)

0

In [10]:
thermo_libs = [
# 'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
# 'PFCA_thermo',
#'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

#kin_families = ['Retroene']
#kin_families = ['F_Abstraction']
#kin_families = '1+2_Cycloaddition'
kin_families = ['R_Addition_MultipleBond']

In [11]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


In [12]:
family = database.kinetics.families[family_to_train]

In [16]:
uncommon_atoms = ['Si']
nodes_to_watch = {}
print('Before regularization:')
for node_label, node in family.groups.entries.items():
    # if node_label in ['Root_N-4R!H->O']:
    #     print(node_label)
    #     print(node.item.to_adjacency_list())
    #     print(node.item.atoms[3].atomtype)
    #     node_of_interest = node


    if all(ua in node.item.to_adjacency_list() for ua in uncommon_atoms):
        nodes_to_watch[node_label] = [node.item.to_adjacency_list()]

Before regularization:


In [14]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([133.883,155.55,173.376,188.387,210.872,224.909,241.381],'J/(mol*K)'), H298=(-1313.31,'kJ/mol'), S298=(422.435,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(241.12,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(O2s-(Cds-O2d)H) + group(CsCFHO) + longDistanceInteraction_noncyclic(Cs(F)-CO) + group(CsFFFO) + group(Cds-OdCsOs) + radical(CsCOF1sO2s)""").
The thermo for this species is probably wrong! Setting CpInf = Cphigh for Entropy calculationat T = 2000.0 K...
ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([133.883,155.55,173.376,188.387,210.872,224.909,241.381],'J/(mol*K)'), H298=(-1313.31,'kJ/mol'), S298=(422.435,'J/(mol*K)'), Cp0=(33.2579,'J/(mol*K)'), CpInf=(241.12,'J/(mol*K)'), comment="""Thermo group additivity estimation: group(O2s-CsCs) + group(O2s-(Cds-O2d)H) + group(CsCFHO) + longDistanceInteraction_noncyclic(Cs(F)-CO) + group(CsFFFO) + group(Cds-OdCsOs) + r

433.1492578983307


In [17]:

for node_label in nodes_to_watch.keys():
    print(node_label)
    print(len(templateRxnMap[node_label]))

Root_3R->H_N-2R!H-inRing_N-2R!H->N_N-1R!H->N_N-2COS->O_N-1COS->O_Sp-2CS=1CCSS_N-2CS->C_Ext-1CS-R_N-4R!H->S
3
